### GCC118 - Programação Matemática
### Universidade Federal de Lavras
#### Instituto de Ciências Exatas e Tecnológicas
##### Profa. Andreza C. Beezão Moreira (DMM/UFLA)
##### Prof. Mayron César O. Moreira (DCC/UFLA)

Seu código deve ser testado na seguinte instância: [link](https://drive.google.com/file/d/12CeZEow-6vVgJFgzXMo0gbjV5hLCM6Zi/view?usp=sharing). O README se encontra em: [link](https://drive.google.com/file/d/1LBTdkDoTQRxUJsKLI4Z38-Uc8oUPCZQ0/view?usp=sharing).

---

## **Alunos**
- Heitor Rodrigues Sabino
- Ranulfo Mascari Neto

---

## **Modelagem Matemática do Problema da Mochila 0/1**

Dado um conjunto de $ n $ itens, onde:

- $ w_i $: peso do item$ i $,
- $ v_i $: valor do item$ i $,
- $ x_i \in \{0, 1\} $: variável de decisão que indica se o item $ i $ foi selecionado ($ x_i = 1 $) ou não ($ x_i = 0 $),

### **Função Objetivo**
$
\text{Maximizar } Z = \sum_{i=1}^n v_i \cdot x_i
$


### **Restrição de Capacidade**
$
\sum_{i=1}^n w_i \cdot x_i \leq C
$
onde $ C $ é a capacidade máxima da mochila.

### **Seleção dos Itens**
A seleção dos itens na solução ótima pode ser representada como:
$
\mathcal{S} = \{ i \in \{1, 2, \dots, n\} \mid x_i^* = 1 \}
$

$ x_i^* $ representa o valor ótimo da variável de decisão para o item $ i $, obtido após a solução do modelo.

---

## **Implementação**

### **Importando e pip**

In [ ]:
!pip install gurobipy

In [ ]:
from gurobipy import Model, GRB
import time

### **Dados**

In [ ]:
def read_input_from_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    weights = []
    values = []

    instance_name = lines[0].strip()
    instance_class = lines[1].strip()

    capacity = int(lines[2].strip())

    for i, line in enumerate(lines[3:], start=4):
        line = line.strip()
        if line:
            weight, value = map(int, line.split())
            weights.append(weight)
            values.append(value)

    return weights, values, capacity


### **Branch and Bound Knapsack**

In [ ]:
def greedy_fractional_knapsack(weights, values, capacity):
    items = sorted([(values[i] / weights[i], weights[i], values[i]) for i in range(len(weights))], reverse=True)
    total_value = 0
    for ratio, weight, value in items:
        if capacity >= weight:
            capacity -= weight
            total_value += value
        else:
            total_value += ratio * capacity
            break
    return total_value

In [ ]:
def branch_and_bound_knapsack(weights, values, capacity):
    start_time = time.time()
    num_items = len(weights)

    model = Model("Knapsack")

    x = model.addVars(num_items, vtype=GRB.BINARY, name="x")

    model.setObjective(sum(values[i] * x[i] for i in range(num_items)), GRB.MAXIMIZE)

    model.addConstr(sum(weights[i] * x[i] for i in range(num_items)) <= capacity, "capacity")

    model.optimize()

    if model.status == GRB.OPTIMAL:
        max_value = model.objVal
        selected_items = [i for i in range(num_items) if x[i].X > 0.5]
    else:
        max_value = None
        selected_items = []

    end_time = time.time()

    fractional_value = greedy_fractional_knapsack(weights, values, capacity)
    gap = (fractional_value - max_value) / fractional_value * 100 if fractional_value > 0 else 0

    return {
        "max_value": max_value,
        "selected_items": selected_items,
        "num_nodes_explored": model.NodeCount,
        "gap": gap,
        "time": end_time - start_time
    }

### **Exibindo solução**

In [ ]:
file_path = "Weakly001"
weights, values, capacity = read_input_from_file(file_path)
result = branch_and_bound_knapsack(weights, values, capacity)
print(f"Max Value: {result['max_value']}")
print(f"Selected Items: {result['selected_items']}")
print(f"Number of Nodes Explored: {result['num_nodes_explored']}")
print(f"GAP: {result['gap']:.2f}%")
print(f"Time: {result['time']:.4f} seconds")

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 1 rows, 1000 columns and 1000 nonzeros
Model fingerprint: 0x9ed7d31e
Variable types: 0 continuous, 1000 integer (1000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [3e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+03, 2e+03]
Found heuristic solution: objective 2709.0000000
Presolve removed 0 rows and 17 columns
Presolve time: 0.03s
Presolved: 1 rows, 983 columns, 983 nonzeros
Variable types: 0 continuous, 983 integer (979 binary)

Root relaxation: objective 4.887475e+03, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  O